# MARK I

#### Packages

In [147]:
from tqdm import tqdm
pd.set_option('max_rows', 1000, 'max_columns', 20)

개요 : 상표 등록과 거절의 feature 찾기
문제정의 : 등록 상표와 거절 상표 사이에서 어떤 의미가 있는가

Step 0. Define    
Step 1. Crawling   
Step 2. EDA, Preprocessing   
Step 3. Modeling    
Step 4. Visualization & Analysis   
Step 5. Feedback   

## Step 0: Define 

## Step 1: Crawling
#### 1. 상표 정보 크롤링      
- Scrapy 를 이용해 크롤링 (진행중)
- 각 데이터의 상표 이미지 url도 포함하여 크롤링

#### 2. url을 이용해 이미지 다운로드   
- 상표 이미지 다운로드

- 이미지 다운로드 전 1차 전처리

In [90]:
df1 = pd.read_csv('../../01_mark/data/fashion.csv')
df2 = pd.read_csv('../../01_mark/data/books.csv')
df3 = pd.read_csv('../../01_mark/data/food.csv')
print(len(df1) + len(df2) + len(df3))

641839


- 사진 파일의 제목과 data 의 indexing 을 맞추어 줄 것이므로, dataframe merge

In [91]:
df = pd.concat([df1, df2, df3])
df = df.reset_index(drop=True)
df.tail()

,date,image_url,status,class1,class2
641834,1965.04.30,http://plus.kipris.or.kr/kiprisplusws/fileToss...,소멸,기타서비스업(112),NaN
641835,1964.10.29,http://plus.kipris.or.kr/kiprisplusws/fileToss...,소멸,기타서비스업(112),NaN
641836,1959.07.10,http://plus.kipris.or.kr/kiprisplusws/fileToss...,포기,기타서비스업(112),NaN
641837,1958.06.24,http://plus.kipris.or.kr/kiprisplusws/fileToss...,포기,기타서비스업(112),NaN
641838,1957.06.10,http://plus.kipris.or.kr/kiprisplusws/fileToss...,무효,기타서비스업(112),NaN


In [32]:
df['date'].isnull().any()

False

- 출원일날짜 형변환

In [92]:
def date_split_year(string_date):
    date_list = string_date.split('.')
    if date_list[0] == ' ':
        return '0'
    else:
        return date_list[0]

def date_split_month(string_date):
    date_list = string_date.split('.')
    try:
        return date_list[1]
    except:
        return '0'

def date_split_day(string_date):
    date_list = string_date.split('.')
    try:
        return date_list[2]
    except:
        return '0'

In [93]:
%time
df = pd.concat([df, df['date'].apply(lambda date: pd.Series({'year': date_split_year(date), \
                                               'month': date_split_month(date),\
                                               'day': date_split_day(date)}))], axis=1)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.11 µs


In [95]:
df['year'] = df['year'].apply(lambda year: int(year))

In [99]:
df['month'] = df['month'].apply(lambda month: int(month))

- 상표법 2016년 전면 개정안이 2016년 9월 1일 자로 시행되므로, 출원일 기준 2016년 9월부터 현재까지의 데이터 만으로 모델링을 할 계획입니다.

In [106]:
df = df[(df['year'] >= 2017) | ((df['year']==2016) & (df['month']>=9))]

In [115]:
df = df.reset_index(drop=True)

In [116]:
df.tail()

,date,image_url,status,class1,class2,year,month,day
83510,2016.09.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,등록,곡물/과자/차(30),요식/호텔업(43),2016,9,12
83511,2016.09.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,등록,요식/호텔업(43),NaN,2016,9,12
83512,2016.09.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,등록,축산/가공식품(29),곡물/과자/차(30),2016,9,12
83513,2016.09.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,등록,요식/호텔업(43),NaN,2016,9,12
83514,2016.09.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,등록,요식/호텔업(43),NaN,2016,9,12


- 모델링에 핵심적으로 사용될 이미지 데이터의 url 이 없는 경우, drop

In [132]:
df = df.drop(index=[28327, 39303, 40745, 41117, 42636, 50766, 52919, 81798])

In [137]:
df = df.drop(columns='index')

In [138]:
df = df.reset_index(drop=True)

In [140]:
df = df.reset_index()
df

,index,date,image_url,status,...,class2,year,month,day
0,0,2019.01.14,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,...,NaN,2019,1,14
1,1,2019.01.13,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,...,NaN,2019,1,13
2,2,2019.01.13,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,...,NaN,2019,1,13
3,3,2019.01.13,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,...,NaN,2019,1,13
4,4,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,...,NaN,2019,1,12
5,5,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,...,NaN,2019,1,12
6,6,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,...,NaN,2019,1,12
7,7,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,...,NaN,2019,1,12
8,8,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,...,NaN,2019,1,12
9,9,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,...,NaN,2019,1,12


In [145]:
df['status'].unique()

array(['출원',
       'http://plus.kipris.or.kr/kiprisplusws/fileToss.jsp?arg=ad7a17eeeef6e4ea4b5e22ef00dd3e293e70a322c3ead7b6c42825a7f354bc2e40717f59b4e44df8e5bfeafe60ea6dfb710543bf315b1c83',
       '공고',
       'http://plus.kipris.or.kr/kiprisplusws/fileToss.jsp?arg=ad7a17eeeef6e4ea4b5e22ef00dd3e293228145e2e03ea140dc336e625d3387b186a2eefb12143fc1673b1be1fd731a64d03d4e52294dea4',
       '거절', '등록',
       'http://plus.kipris.or.kr/kiprisplusws/fileToss.jsp?arg=ad7a17eeeef6e4ea4b5e22ef00dd3e293228145e2e03ea14bebf1537515e159bd0b14af852a36e6d8d4d22c18198538c65f19ee6f72006d0',
       'http://plus.kipris.or.kr/kiprisplusws/fileToss.jsp?arg=ad7a17eeeef6e4ea4b5e22ef00dd3e293228145e2e03ea149fd5e85681d4b10b03474dafc17864b1b4c8d7e9e48b163e76727b35aae294a2',
       '취하', '포기',
       'http://plus.kipris.or.kr/kiprisplusws/fileToss.jsp?arg=ad7a17eeeef6e4ea4b5e22ef00dd3e293228145e2e03ea14cf545deb97a896f6ea0744bf9f603e750f22be51f026270f35f6d59dc3f6a495',
       '2018.03.05',
       'http://plus.kipri

In [148]:
df[df['status']=='2018.03.05']

,index,date,image_url,status,class1,class2,year,month,day
13657,13657,2018.03.01,2018.02.27,2018.03.05,http://plus.kipris.or.kr/kiprisplusws/fileToss...,등록,2018,3,01
67121,67121,2018.03.01,2018.02.27,2018.03.05,http://plus.kipris.or.kr/kiprisplusws/fileToss...,등록,2018,3,01


In [150]:
df = df.drop(index=[13657, 67121])
df = df.drop(columns='index')
df = df.reset_index(drop=True)
df

,date,image_url,status,class1,class2,year,month,day
0,2019.01.14,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,14
1,2019.01.13,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,13
2,2019.01.13,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,13
3,2019.01.13,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,13
4,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12
5,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12
6,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12
7,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12
8,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12
9,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12


In [152]:
df = df[(df['status']=='출원')|(df['status']=='공고')|(df['status']=='거절')|(df['status']=='등록')|(df['status']=='취하')|\
  (df['status']=='포기')]
df = df.reset_index(drop=True)
df

,date,image_url,status,class1,class2,year,month,day
0,2019.01.14,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,14
1,2019.01.13,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,13
2,2019.01.13,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,13
3,2019.01.13,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,13
4,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12
5,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12
6,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12
7,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12
8,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12
9,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12


In [167]:
df['status'].value_counts()

등록    30903
출원    26573
공고    11831
거절     8891
포기     5061
취하       72
Name: status, dtype: int64

- 상표법 제57조 제1항 및 제2항에서 “심사관은 상표등록출원에 대하여 거절이유를 발견할 수 없는 때에는 출원공고를 하여야 하며, 특허청장은 출원공고결정이 있을 경우에는 그 결정의 등본을 출원인에게 송달하고, 그 상표등록출원에 관하여 상표공보에 게재하여 출원공고를 하여야 한다.”고 규정하고 있기 때문에 출원인에게 출원공고결정서를 통지하고 있습니다.

In [168]:
df['class1'].value_counts()

요식/호텔업(43)        50592
광학/통신기기/SW(09)     8634
교육/엔터테인먼트업(41)     8427
종이/인쇄물(16)         6049
축산/가공식품(29)        2359
곡물/과자/차(30)        2332
광고/도소매업(35)        2291
화장품/세제(03)          546
보험/금융업(36)          216
운송/보관업(39)          213
의류/신발/모자(25)        150
맥주/음료수(32)          147
통신/방송업(38)          129
의약품류/위생제품(05)       113
귀금속(14)             112
화학제품/비료(01)         111
가정용품/유리제품(21)       103
기계/모터/엔진(07)         86
금속제품(06)             84
가죽제품(18)             70
천연/농산품(31)           66
완구/스포츠용구(28)         60
가공업(40)              59
냉난방/조명/가전(11)        57
주류(33)               50
가구/목제(20)            46
컴퓨터/SW개발업(42)        44
건설/수리업(37)           40
페인트/잉크(02)           25
윤활유/연료(04)           22
공구/면도기(08)           19
자동차/수송기구(12)         15
의료기기(10)             15
직물제품(24)             12
흡연제품(34)              8
법무/보안서비스업(45)         7
비금속건축자재(19)           6
레이스/리본(26)            6
의료/뷰티/농업(44)          4
고무제품(17)              4
로프/포장용품(22)           2
Name: class1, dt

In [165]:
len(df[df['class1']=='요식/호텔업(43)'])

50592

In [164]:
len(df[df['class1']=='교육/엔터테인먼트업(41)'])

8427

In [166]:
len(df[df['class1']=='종이/인쇄물(16)'])

6049

In [169]:
df

,date,image_url,status,class1,class2,year,month,day
0,2019.01.14,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,14
1,2019.01.13,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,13
2,2019.01.13,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,13
3,2019.01.13,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,13
4,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12
5,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12
6,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12
7,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12
8,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12
9,2019.01.12,http://plus.kipris.or.kr/kiprisplusws/fileToss...,출원,요식/호텔업(43),NaN,2019,1,12


In [170]:
df.to_csv('../../01_mark/data/total.csv', index=True)